In [162]:
!pip install chromadb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [163]:
!pip install -U langchain-chroma

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [164]:
import os
import pysqlite3
import sys
sys.modules["sqlite3"] = sys.modules.pop("pysqlite3")
import chromadb
from chromadb.config import Settings

In [165]:
import gradio as gr
from llama_index.llms.ollama import Ollama
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.llms.base import LLM

from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
#from langchain_community.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain.vectorstores import VectorStore
from typing import List, Any
from chromadb import Client
from langchain.chains import LLMChain, ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain

In [166]:
chroma_client = chromadb.Client()

Failed to send telemetry event ClientStartEvent: module 'chromadb' has no attribute 'get_settings'


In [167]:
# switch `create_collection` to `get_or_create_collection` to avoid creating a new collection every time
collection = chroma_client.get_or_create_collection(name="dwarf_collect")

Failed to send telemetry event ClientCreateCollectionEvent: module 'chromadb' has no attribute 'get_settings'


In [168]:
# Define a custom LangChain wrapper for the Ollama LLaMA model
class OllamaLangChain(LLM):
    model_name: str = "llama3.2:latest"  # Declared field
    request_timeout: int = 90           # Declared field

    def __init__(self, model_name="llama3.2:latest", request_timeout=30):
        super().__init__()
        self.model_name = model_name
        self.request_timeout = request_timeout
        # Use object.__setattr__ to bypass Pydantic's field validation
        object.__setattr__(self, "_client", Ollama(model=model_name, request_timeout=request_timeout))
    
    @property
    def _llm_type(self):
        return "ollama"

    def _call(self, prompt, stop=None):
        # Access the private client
        client = object.__getattribute__(self, "_client")
        response = client.complete(prompt)
        return response.text

In [169]:
# Read the text file
with open("docs/dwarf.txt", "r", encoding="utf-8") as file:
    text = file.read()

# Create a list of a single document
documents = [Document(page_content=text)]

In [170]:
embeddings = OllamaEmbeddings(
    model="llama3.2"
)

In [171]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 200,
    chunk_overlap  = 16
    
)
# Split the text into chunks
splitted_txt = text_splitter.split_documents(documents)

In [172]:
# Output the result
for chunk in splitted_txt:
    print(chunk.page_content)
    print("\n")

Dwarf
Dwarf Traits
Your dwarf character has an assortment of inborn
abilities, part and parcel of dwarven nature.
Ability Score Increase. Your Constitution score
increases by 2.


increases by 2.
Age. Dwarves mature at the same rate as humans,
but they’re considered young until they reach the
age of 50. On average, they live about 350 years.


Alignment. Most dwarves are lawful, believing
firmly in the benefits of a well-­‐‑ordered society. They
tend toward good as well, with a strong sense of fair


play and a belief that everyone deserves to share in
the benefits of a just order.
Size. Dwarves stand between 4 and 5 feet tall and
average about 150 pounds. Your size is Medium.


Speed. Your base walking speed is 25 feet. Your
speed is not reduced by wearing heavy armor.
Darkvision. Accustomed to life underground, you
have superior vision in dark and dim conditions. You


can see in dim light within 60 feet of you as if it were
bright light, and in darkness as if it were dim light.
You 

In [173]:
vectorstore = Chroma(
    embedding_function=embeddings,
    collection_name="dwarf_collect",
    client=chroma_client
)


Failed to send telemetry event ClientCreateCollectionEvent: module 'chromadb' has no attribute 'get_settings'


In [174]:
# Add documents to the collection
vectorstore.add_documents(splitted_txt)

['a168d486-aab3-4dda-b452-904624010655',
 'b9e6d8ab-46bd-4385-9906-9127fc2485b6',
 '855b4a71-ca80-4095-99ed-2445f6a735e2',
 '10e77fd7-d87e-4d19-b90f-87034140f06f',
 '5de93047-54ea-4c10-bbf9-e3bbf038d077',
 '5fbf7c4c-c66f-4a24-8efd-11200d508357',
 '6f99eb00-5568-4d2d-a136-5bef08e01981',
 'c48b20ed-3910-46bb-8838-6a6d52533094',
 '75b1dfd2-1a49-431b-8881-9ae6e72a2ea9',
 '7eaa4943-428e-488b-b665-8fa09582e700',
 '2ac170fe-04ac-4a23-adde-5c19dc91dc13',
 '9b5f8790-fbab-4163-b356-1c03c53bd6e7',
 '91e909be-5582-4d6c-84bd-e619c7e648d4']

In [183]:
retriever = vectorstore.as_retriever()

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
Given the context: [Document(page_content='play and a belief that everyone deserves to share in\nthe benefits of a just order.\nSize. Dwarves stand between 4 and 5 feet tall and\naverage about 150 pounds. Your size is Medium.'), Document(page_content='play and a belief that everyone deserves to share in\nthe benefits of a just order.\nSize. Dwarves stand between 4 and 5 feet tall and\naverage about 150 pounds. Your size is Medium.'), Document(page_content='play and a belief that everyone deserves to share in\nthe benefits of a just order.\nSize. Dwarves stand between 4 and 5 feet tall and\naverage about 150 pounds. Your size is Medium.'), Document(page_content='increases by 1, and it increases by 1 every time you\ngain a level.')], answer the question: I am a Dwarf, what about Agility ?


In [176]:
llm = OllamaLangChain()
memory = ConversationBufferMemory()  # Store chat history


In [177]:
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)

In [178]:
prompt_template = PromptTemplate(
    input_variables=["question", "context"],  # context here is the document content
    template="Given the context: {context}, answer the question: {question}"
)
# Create the LLMChain for the question generation
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    memory=memory,
    verbose=True
)

In [179]:
# Define the combine_docs_chain with document_variable_name 'context'
combine_docs_chain = load_qa_chain(
    llm=llm,
    chain_type='stuff',
    document_variable_name='context',  # Ensure this matches the input in the LLMChain
    verbose=True
)

# Create the ConversationalRetrievalChain with the manually created question generator chain
retrieval_chain = ConversationalRetrievalChain(
    retriever=retriever,  # Your retriever instance
    question_generator=llm_chain,  # Use the LLMChain as question generator
    combine_docs_chain=combine_docs_chain,  # Chain to combine the documents
    memory=memory,  # Memory to store conversation history
    verbose=True  # Enable verbosity for debugging
)

In [180]:
# Chat function
"""def chat_with_langchain(input_text):
    # Use LangChain's ConversationChain
    response = conversation.run(input_text)
    return response"""

"def chat_with_langchain(input_text):\n    # Use LangChain's ConversationChain\n    response = conversation.run(input_text)\n    return response"

In [190]:
# Define a chat function for Gradio
def chat_with_langchain(input_text):
    # Check if the input contains any document-related query
    
    # Retrieve relevant documents based on input query
    context = retriever.get_relevant_documents(input_text)  # Get the relevant document context
    question = input_text  # Use the entire input as the question
    
    # Ensure you only pass the 'question' key to the LLM chain
    response = llm_chain.run({"question": question})
    
    return response




In [191]:
# Gradio interface
interface = gr.Interface(
    fn=chat_with_langchain,
    inputs="text",
    outputs="text",
    title="LLaMA 3 Chatbot with LangChain",
    description="Chat with a LLaMA 3-based model via Ollama and LangChain!"
)
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7868
Running on public URL: https://100d4fcba4129f14ca.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Traceback (most recent call last):
  File "/home/capmars/SemAgent/.venv/lib/python3.11/site-packages/gradio/queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/capmars/SemAgent/.venv/lib/python3.11/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/capmars/SemAgent/.venv/lib/python3.11/site-packages/gradio/blocks.py", line 1935, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/capmars/SemAgent/.venv/lib/python3.11/site-packages/gradio/blocks.py", line 1520, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^